In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np


In [2]:

import pandas as pd

In [4]:


# Load your data
df = pd.read_csv("../Datasets Clase 2/tb_sellout_01_todos.csv")
df


,Unnamed: 0,periodo,product_id,cust_request_qty,cust_request_tn,tn
0,0,201701,20001,479.0,937.72717,934.77222
1,1,201701,20002,391.0,555.18654,550.15707
2,2,201701,20003,438.0,1067.81543,1063.45835
3,3,201701,20004,339.0,569.37394,555.91614
4,4,201701,20005,249.0,494.60084,494.27011
...,...,...,...,...,...,...
44383,44383,201912,21247,37.0,0.14544,0.14544
44384,44384,201912,20728,98.0,14.31612,14.31612
44385,44385,201912,20770,6.0,3.18500,3.18500
44386,44386,201912,20792,7.0,3.80934,3.80934


In [ ]:

# Create an empty DataFrame to store the final results
final_lstm_predictions = pd.DataFrame(columns=['product_id', 'predicted_tn_lstm', 'real_tn_lstm'])

# Loop through each unique product_id to fit a separate LSTM model
for product in df['product_id'].unique():
    # Filter data for the specific product
    product_data = df[df['product_id'] == product].sort_values('periodo')
    
    # Prepare data
    X = product_data[['cust_request_qty', 'cust_request_tn']].values
    y = product_data['tn'].values
    
    # Reshape data for LSTM
    X = np.reshape(X, (X.shape[0], 1, X.shape[1]))
    
    # Define the LSTM model
    model = Sequential()
    model.add(LSTM(50, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    
    # Fit the model
    model.fit(X, y, epochs=50, batch_size=1, verbose=0)
    
    # Make prediction for period 201904 (assuming you have the features for it)
    cust_request_qty_201904 = product_data[product_data['periodo'] == 201904]['cust_request_qty'].values[0]
    cust_request_tn_201904 = product_data[product_data['periodo'] == 201904]['cust_request_tn'].values[0]
    X_predict = np.array([[cust_request_qty_201904, cust_request_tn_201904]])  # Replace these with actual values
    X_predict = np.reshape(X_predict, (X_predict.shape[0], 1, X_predict.shape[1]))
    
    forecast_lstm = model.predict(X_predict)
    
    # Append the prediction to the final DataFrame
    final_lstm_predictions = final_lstm_predictions.append({'product_id': product, 'predicted_tn_lstm': forecast_lstm[0][0]}, ignore_index=True)

# Merge the LSTM predictions with the actual values for 201904
final_lstm_predictions = final_lstm_predictions.merge(df[df['periodo'] == 201904][['product_id', 'tn']], on='product_id', how='left')
final_lstm_predictions.rename(columns={'tn': 'actual_tn'}, inplace=True)

# Save to CSV
final_lstm_predictions.to_csv('predictions_with_actual_lstm.csv', index=False)


In [1]:
from empresa4.datasets.test import test_dataset_is_here
test_dataset_is_here()

"IT'S A MIRABLE!"